**The main content of this article will present how the AlexNet Convolutional Neural Network(CNN) architecture is implemented using TensorFlow and Keras.**

In [ ]:
import keras
from keras.datasets import cifar10
from keras import backend as K
from keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dense, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model
from keras.layers import concatenate,Dropout,Flatten

from keras import optimizers,regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import he_normal
from keras.callbacks import LearningRateScheduler, TensorBoard, ModelCheckpoint

num_classes        = 10
batch_size         = 64
iterations         = 782
epochs             = 300
DROPOUT=0.5
CONCAT_AXIS=3
weight_decay=1e-4
DATA_FORMAT='channels_last'
log_filepath  = './alexnet'

In [ ]:
def color_preprocessing(x_train,x_test):
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    mean = [125.307, 122.95, 113.865]
    std  = [62.9932, 62.0887, 66.7048]
    for i in range(3):
        x_train[:,:,:,i] = (x_train[:,:,:,i] - mean[i]) / std[i]
        x_test[:,:,:,i] = (x_test[:,:,:,i] - mean[i]) / std[i]
    return x_train, x_test

def scheduler(epoch):
    if epoch < 100:
        return 0.01
    if epoch < 200:
        return 0.001
    return 0.0001

# load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test  = keras.utils.to_categorical(y_test, num_classes)
x_train, x_test = color_preprocessing(x_train, x_test)

170498071/170498071 [==============================] - 2s 0us/step


In [ ]:
def alexnet(img_input,classes=10):
    x = Conv2D(96,(11,11),strides=(4,4),padding='same',
               activation='relu',kernel_initializer='uniform')(img_input)# valid
    x = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same',data_format=DATA_FORMAT)(x)

    x = Conv2D(256,(5,5),strides=(1,1),padding='same',
               activation='relu',kernel_initializer='uniform')(x)
    x = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same',data_format=DATA_FORMAT)(x)

    x = Conv2D(384,(3,3),strides=(1,1),padding='same',
               activation='relu',kernel_initializer='uniform')(x)

    x = Conv2D(384,(3,3),strides=(1,1),padding='same',
               activation='relu',kernel_initializer='uniform')(x)

    x = Conv2D(256,(3,3),strides=(1,1),padding='same',
               activation='relu',kernel_initializer='uniform')(x)
    x = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same',data_format=DATA_FORMAT)(x)
    x = Flatten()(x)
    x = Dense(4096,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096,activation='relu')(x)
    x = Dropout(0.5)(x)
    out = Dense(classes, activation='softmax')(x)
    return out

In [ ]:
img_input=Input(shape=(32,32,3))
output = alexnet(img_input)
model=Model(img_input,output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 8, 8, 96)          34944     
                                                                 
 max_pooling2d (MaxPooling2  (None, 4, 4, 96)          0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 4, 256)         614656    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 2, 2, 256)         0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 2, 2, 384)         885120

In [ ]:
# set optimizer
sgd = optimizers.SGD(learning_rate=.1, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# set callback
tb_cb = TensorBoard(log_dir=log_filepath, histogram_freq=0)
change_lr = LearningRateScheduler(scheduler)
cbks = [change_lr,tb_cb]

# set data augmentation
datagen = ImageDataGenerator(horizontal_flip=True,
                             width_shift_range=0.125,
                             height_shift_range=0.125,
                             fill_mode='constant',cval=0.)

#horizontal_flip=True Randomly flip inputs horizontally.
# width_shift_range=0.125
#height_shift_range=0.125
#fill_mode='constant',cval=0 'constant': kkkkkkkk|abcd|kkkkkkkk (cval=k)
datagen.fit(x_train)

# start training
model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),
                    steps_per_epoch=iterations,
                    epochs=epochs,
                    callbacks=cbks,
                    validation_data=(x_test, y_test))
model.save('alexnet.h5')

<ipython-input-6-8f2c7a66eecc>:23: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),


Epoch 1/300
782/782 [==============================] - 38s 42ms/step - loss: 1.7030 - accuracy: 0.3667 - val_loss: 1.4709 - val_accuracy: 0.4765 - lr: 0.0100
Epoch 2/300
782/782 [==============================] - 31s 40ms/step - loss: 1.4068 - accuracy: 0.4935 - val_loss: 1.2931 - val_accuracy: 0.5464 - lr: 0.0100
Epoch 3/300
782/782 [==============================] - 32s 41ms/step - loss: 1.2784 - accuracy: 0.5429 - val_loss: 1.1580 - val_accuracy: 0.5978 - lr: 0.0100
Epoch 4/300
782/782 [==============================] - 31s 40ms/step - loss: 1.1956 - accuracy: 0.5738 - val_loss: 1.1225 - val_accuracy: 0.6162 - lr: 0.0100
Epoch 5/300
782/782 [==============================] - 31s 39ms/step - loss: 1.1325 - accuracy: 0.6012 - val_loss: 1.0406 - val_accuracy: 0.6377 - lr: 0.0100
Epoch 6/300
782/782 [==============================] - 32s 41ms/step - loss: 1.0734 - accuracy: 0.6228 - val_loss: 1.0441 - val_accuracy: 0.6372 - lr: 0.0100
Epoch 7/300
782/782 [==============================]